# Import Necessary Library 

In [1]:
# importing libraries
import pandas as pd
import scipy
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from os import path
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Import Dataset

In [2]:
df_raw = pd.read_csv("/kaggle/input/mitm-attack/MitM.csv", low_memory= False)
df_raw.head()

KeyboardInterrupt: 

In [ ]:
df_raw.shape

In [ ]:
df_raw.info()

In [ ]:
df_raw.isnull().sum()

In [ ]:
df_raw.describe()

In [ ]:
df_raw['type'].value_counts()

In [ ]:
# Drop columns with too many missing values (e.g., 50% or more)
threshold = 0.5
df = df_raw.copy()
df = df.loc[:, df.isnull().mean() < threshold]

# Impute remaining null values using median for numerical columns
num_cols = df.select_dtypes(include=[np.number]).columns
imputer = SimpleImputer(strategy='median')
df[num_cols] = imputer.fit_transform(df[num_cols])

# Forward fill for categorical columns (if applicable)
cat_cols = df.select_dtypes(include=[object]).columns
df[cat_cols] = df[cat_cols].ffill()


In [ ]:
df['type'] = df['type'].map({'normal': 1, 'mitm': 0})

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# Select only numeric columns for correlation analysis
numeric_df = df.copy()
num_col = numeric_df.select_dtypes(include=[np.number]).columns
data_num = df[num_col].copy()
data_num.head()

In [ ]:
data_num.shape

In [ ]:
cat_col = df.columns.difference(num_col)
cat_col = cat_col[1:]
cat_col

In [ ]:
# creating a dataframe with only categorical attributes
data_cat = df_raw[cat_col].copy()
data_cat.head()

In [ ]:
data_num

In [ ]:
# Correlation analysis for numeric features pertimbangkan untuk drop yang korelasinya rendah
corr_matrix = data_num.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()

In [ ]:
data_cat = pd.get_dummies(data_cat,columns=cat_col)

In [ ]:
data_cat.head()

In [ ]:
df = pd.concat([df, data_cat],axis=1)

In [ ]:
df.columns

In [ ]:
df.drop(columns=cat_col,inplace=True)

# Data Normalization

In [ ]:
num_col = list(df.select_dtypes(include='number').columns)
num_col.remove('type')

In [ ]:
num_col

In [ ]:
minmax_scale = MinMaxScaler(feature_range=(0, 1))

In [ ]:
def normalization(df,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    df[i] = minmax_scale.fit_transform(arr.reshape(len(arr),1))
  return df

In [ ]:
df.head()

In [ ]:
df_normalization = df.copy()
df_normalization = normalization(df.copy(),num_col)

In [ ]:
df_normalization = df_normalization.drop(columns=['eth.dst'])

In [ ]:
df_normalization.columns

In [ ]:
df_normalization['type'].value_counts()

In [ ]:
# label
bin_label = pd.DataFrame(df_normalization['type'])

In [ ]:
bin_label.value_counts()

In [ ]:
# creating a dataframe with binary labels (normal,abnormal)
bin_data = df_normalization.copy()

# Menambahkan kolom 'type' dengan bin_label
bin_data['type'] = bin_label

In [ ]:
bin_data['type'].value_counts()

In [ ]:
import pandas as pd

# Contoh: Salin DataFrame normalisasi
bin_data = df_normalization.copy()

# Pastikan bin_label adalah Series atau list dengan panjang yang sesuai
if isinstance(bin_label, (list, np.ndarray)):
    bin_label = pd.Series(bin_label)

# Cek apakah panjang bin_label sama dengan bin_data
if len(bin_data) == len(bin_label):
    bin_data['type'] = bin_label
else:
    print("Panjang bin_label tidak sesuai dengan jumlah baris bin_data.")

# Menampilkan beberapa baris untuk memeriksa
print(bin_data.head())
print(bin_data.columns)  # Memeriksa semua kolom dalam bin_data


In [ ]:
bin_label = bin_label.replace({1: 'normal', 0: 'MitM'})

In [ ]:
bin_label.head()

In [ ]:
# label encoding (0,1) binary labels
le1 = preprocessing.LabelEncoder()
enc_label = bin_label.apply(le1.fit_transform)
bin_data['type'] = enc_label

In [ ]:
le1.classes_

In [ ]:
bin_label.value_counts()

# Data Visualization

In [ ]:
plt.figure(figsize=(8,8))
plt.pie(df_raw.type.value_counts(),labels=['normal','MitM'],autopct='%0.2f%%')
plt.title("Pie chart distribution of normal and abnormal labels",fontsize=16)
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the distribution of the labels
plt.figure(figsize=(6, 4))
sns.countplot(x='type', data=df_raw)
plt.title('Distribution of Labels')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()


# SMOTE

Dikarenakan jauhnya perbedaan pada distribusi label, kita akan menyamaratakan labelnya menggunakan metode SMOTE

In [ ]:
# 1. Mengimpor Modul yang Diperlukan
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from os import path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score


In [ ]:
bin_data.head()

In [ ]:
X = bin_data.drop(columns=['type'],axis=1)
y = bin_data['type']

In [ ]:
# # changing attack labels into two categories 'normal' and 'mitm'
# y_bin = y_bin.replace({1: 'normal', 0: 'MitM'})

In [ ]:
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X, y, test_size=0.2, random_state=50)

In [ ]:
y_test_bin

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
y.value_counts()

In [ ]:
X_train_bin

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_bin, y_train_bin)

from collections import Counter
print(f"Distribusi kelas sebelum SMOTE: {Counter(y_train_bin)}")
print(f"Distribusi kelas setelah SMOTE: {Counter(y_resampled)}")

In [ ]:
X_resampled

In [ ]:
y_resampled.value_counts()

In [ ]:
plt.figure(figsize=(8,8))
plt.pie(y_resampled.value_counts(),labels=['Normal','MitM'],autopct='%0.2f%%')
plt.title("Pie chart distribution of normal and abnormal labels",fontsize=16)
plt.legend()
# plt.savefig('/content/gdrive/My Drive/ASDOS/template/Topik1_Studi Transformer-Based Sequence Modelling untuk Deteksi Ransomware Melalui Analisis Trafik Jaringan yang Mengenkripsi Secara Tiba-tiba/plot/Pie_chart_binary2.png')
plt.show()

In [ ]:
bin_data

# Import Transformer

In [ ]:
!pip install transformers tensorflow

# Transformer Encoder

In [ ]:
le1.classes_

In [ ]:
class_names = le1.classes_

In [ ]:
le1.classes_

# Training and Validation Data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Preprocessing: Label Encoding dan Normalization
# le = LabelEncoder()
# y_bin = le.fit_transform(combined_df['type'].values)  # Encode labels
class_names = le1.classes_  # Save the original class names

# Normalize feature data
scaler = StandardScaler()
X_bin = scaler.fit_transform(combined_df.drop(columns=['type']).values)
y_bin = combined_df['type'].values

# 2. Define Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, 64)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=64, nhead=2, batch_first=True),
            num_layers=2
        )
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add sequence dimension (batch_size, seq_length, feature_dim)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)  # Average pooling
        return self.fc(x)

# 3. KFold Cross-Validation
kf = KFold(n_splits=10, shuffle=True, random_state=50)
all_accuracy, all_f1_scores, all_recall_scores = [], [], []
conf_matrix_list = []

for fold, (train_index, val_index) in enumerate(kf.split(X_bin)):
    print(f'Fold {fold + 1}')

    # Split data
    X_train, X_val = X_bin[train_index], X_bin[val_index]
    y_train, y_val = y_bin[train_index], y_bin[val_index]

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.LongTensor(y_train)
    X_val_tensor = torch.FloatTensor(X_val)
    y_val_tensor = torch.LongTensor(y_val)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

    # Initialize the model
    model = TransformerModel(input_dim=X_train.shape[1], num_classes=len(class_names))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # Early stopping setup
    best_accuracy = 0
    patience, trigger_times = 5, 0

    # Lists for storing loss and accuracy values for each epoch
    train_losses, train_accuracies, val_losses, val_accuracies = [], [], [], []
    num_epochs = 50
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # Training loop
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)
        
        scheduler.step()
        
        # Calculate average training loss and accuracy for this epoch
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        train_accuracy = correct_train / total_train
        train_accuracies.append(train_accuracy)

        # Validate the model
        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            y_pred_probs = model(X_val_tensor)
            val_loss = criterion(y_pred_probs, y_val_tensor).item()
            val_running_loss += val_loss
            _, y_pred = torch.max(y_pred_probs, 1)
        
        # Append validation loss and accuracy
        val_losses.append(val_running_loss)
        accuracy = accuracy_score(y_val, y_pred.numpy())
        val_accuracies.append(accuracy)

        # Save the best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), f'model_fold_{fold + 1}.pt')
            trigger_times = 0
        else:
            trigger_times += 1

        if trigger_times >= patience:
            print(f'Early stopping triggered at epoch {epoch + 1}')
            break

    # Metrics calculation
    conf_matrix = confusion_matrix(y_val, y_pred.numpy())
    conf_matrix_list.append(conf_matrix)
    all_accuracy.append(accuracy)

    f1 = f1_score(y_val, y_pred.numpy(), average='weighted')
    all_f1_scores.append(f1)

    recall = recall_score(y_val, y_pred.numpy(), average='weighted')
    all_recall_scores.append(recall)

    # Print results for the current fold
    print(f'Accuracy for fold {fold + 1}: {accuracy * 100:.2f}%')
    print(f'Recall for fold {fold + 1}: {recall:.4f}')
    print(f'F1 Score for fold {fold + 1}: {f1:.4f}')
    cls_report = classification_report(y_val, y_pred.numpy(), target_names=le1.classes_)
    print(cls_report)

    # Plot loss and accuracy for the current fold
    plt.figure(figsize=(14, 10))
    
    # Plot for training and validation loss
    plt.subplot(2, 1, 1)
    plt.plot(train_losses, label="Training Loss")
    plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f'Loss for Fold {fold + 1}')
    plt.legend()
    
    # Plot for training and validation accuracy
    plt.subplot(2, 1, 2)
    plt.plot(train_accuracies, label="Training Accuracy")
    plt.plot(val_accuracies, label="Validation Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f'Accuracy for Fold {fold + 1}')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

    # Plot confusion matrix for the current fold
    plt.figure(figsize=(6, 4))
    sns.heatmap(conf_matrix, annot=True, fmt='.0f', cmap='Blues', cbar=False,
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix for Fold {fold + 1}')
    plt.show()

# 4. Average Metrics Across All Folds
print(f'Average Accuracy: {np.mean(all_accuracy) * 100:.2f}%')
print(f'Average Recall: {np.mean(all_recall_scores):.4f}')
print(f'Average F1 Score: {np.mean(all_f1_scores):.4f}')

# Plot accuracy, recall, and F1 score across all folds
folds = np.arange(1, len(all_accuracy) + 1)

# Plot Accuracy per fold
plt.figure(figsize=(10, 6))
plt.plot(folds, np.array(all_accuracy) * 100, marker='o', label='Accuracy (%)')
plt.xlabel('Fold')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy per Fold')
plt.xticks(folds)  # Ensure x-axis corresponds to fold numbers
plt.legend()
plt.grid(True)
plt.show()

# Plot Recall per fold
plt.figure(figsize=(10, 6))
plt.plot(folds, all_recall_scores, marker='o', label='Recall')
plt.xlabel('Fold')
plt.ylabel('Recall')
plt.title('Recall per Fold')
plt.xticks(folds)  # Ensure x-axis corresponds to fold numbers
plt.legend()
plt.grid(True)
plt.show()

# # Plot F1 Score per fold
# plt.figure(figsize=(10, 6))
# plt.plot(folds, all_f1_scores, marker='o', label='F1 Score')
# plt.xlabel('Fold')
# plt.ylabel('F1 Score')
# plt.title('F1 Score per Fold')
# plt.xticks(folds)  # Ensure x-axis corresponds to fold numbers
# plt.legend()
# plt.grid(True)
# plt.show()



# Test Data

In [ ]:
# 5. Testing
# Assume `X_test` dan `y_test` adalah dataset testing yang sudah dipersiapkan sebelumnya
# X_test adalah data fitur dan y_test adalah label yang sesuai

# Normalize test data using the same scaler
X_test = scaler.transform(X_test)

# Convert to PyTorch tensor
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test.values)

# Create DataLoader for the test dataset
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Initialize the model (load the best model from the fold with highest accuracy)
best_fold = np.argmax(all_accuracy)  # Get the fold with the highest accuracy
model_path = f'model_fold_{best_fold + 1}.pt'
model.load_state_dict(torch.load(model_path))

# Put model in evaluation mode
model.eval()

# Testing loop
test_running_loss = 0.0
correct_test = 0
total_test = 0
y_test_pred = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_running_loss += loss.item()
        
        # Calculate test accuracy
        _, predicted = torch.max(outputs, 1)
        correct_test += (predicted == labels).sum().item()
        total_test += labels.size(0)
        
        y_test_pred.extend(predicted.numpy())

# Calculate average test loss and accuracy
avg_test_loss = test_running_loss / len(test_loader)
test_accuracy = correct_test / total_test

# Print test results
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Loss: {avg_test_loss:.4f}')

# Classification report on test set
print('Test Classification Report:')
print(classification_report(y_test, y_test_pred, target_names=class_names))

# Confusion matrix for the test set
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix_test, annot=True, fmt='.0f', cmap='Blues', cbar=False,
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Test Set')
plt.show()

# Optionally, plot metrics for test data similar to training/validation
